In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


import nltk
nltk.download('punkt')

import nltk
nltk.download('wordnet')

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
# data_file = open('intents.json').read()
# intents = json.loads(data_file)

with open(r"/content/drive/MyDrive/Colab Notebooks/NLP_2/json_data.json","r") as json_data:
    intents = json.load(json_data)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
# classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

####epochs
no_epochs=200
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=no_epochs, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")



import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random


with open(r"/content/drive/MyDrive/Colab Notebooks/NLP_2/json_data.json","r") as json_data:
    intents = json.load(json_data)

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
from collections import defaultdict

dic=defaultdict(str)


qs=["Hi","Hello","Good Morning"]
dic["Hi"]="Hi, I am ChatBot, How can I help you?"
dic["Hello"]="Hello, I am ChatBot, How can I help you?"
dic["Good Morning"]="Good Morning, I wish you a good day, How can I help you?"


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # print(sentence_words)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    # print("Model result: ", res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    # print("results are: ", results)
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            # print("our tag ",tag)
            # print("i: ", i['tag'], i['responses'])
            # print("i[responses]: ")
            # print(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    # print("ints are: ", ints)
    if len(ints)==0:
      return "Sorry, I didn't get you, can you please ask in another way?"
    res = getResponse(ints, intents)
    return res


print("---------------------------------Chtabot starts----------------------\n")
print("------------------------If you want to stop chatting, enter exit-------------\n")
while True:
  s=input()
  if s=='exit':
    break
  if s in qs:
    print(dic[s])
  else:
    s=s.strip()
    res=chatbot_response(s)
    print(res)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
206 documents
110 classes [['incubation_corona'], ['corona'], ['symptoms_corona'], ['vaccine'], ['treatment_corona'], ['protect'], ['worker_risk'], ['WHO'], ['risk_corona'], ['test_doctor'], ['covid_SARS'], ['SARS'], ['transmission_corona'], ['infectious'], ['infection'], ['risk'], ['risk_transmission'], ['risk_pregnant'], ['test_covid'], ['test'], ['allergy_covid'], ['allergy_symptoms'], ['avoid_infecting'], ['distancing'], ['mask_covid'], ['covid_influenza'], ['europe_covid'], ['EU'], ['peak'], ['schools'], ['restriction'], ['travel'], ['food'], ['animal'], ['lung'], ['measure'], ['isolate'], ['safe'], ['covid'], ['fever'], ['mask'], ['smoke'], ['condition'], ['flu_vaccine'], ['vulnerable'], ['Europe'], ['faces'], ['catch'], ['worry'], ['antibiotic'], ['medicine

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


42/42 [==============================] - 0s 2ms/step - loss: 4.6860 - accuracy: 0.0068
Epoch 2/200
42/42 [==============================] - 0s 2ms/step - loss: 4.5536 - accuracy: 0.0519
Epoch 3/200
42/42 [==============================] - 0s 2ms/step - loss: 4.2319 - accuracy: 0.1208
Epoch 4/200
42/42 [==============================] - 0s 2ms/step - loss: 4.2286 - accuracy: 0.1059
Epoch 5/200
42/42 [==============================] - 0s 2ms/step - loss: 3.8150 - accuracy: 0.1584
Epoch 6/200
42/42 [==============================] - 0s 2ms/step - loss: 3.7609 - accuracy: 0.1981
Epoch 7/200
42/42 [==============================] - 0s 2ms/step - loss: 3.4943 - accuracy: 0.2428
Epoch 8/200
42/42 [==============================] - 0s 2ms/step - loss: 3.2435 - accuracy: 0.2549
Epoch 9/200
42/42 [==============================] - 0s 2ms/step - loss: 2.9475 - accuracy: 0.3427
Epoch 10/200
42/42 [==============================] - 0s 2ms/step - loss: 3.0139 - accuracy: 0.3023
Epoch 11/200
42/42 [=